In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt

# get uavsar VV coherence, int, unw, inc, dem

In [2]:
data_dir = Path('/bsuhome/zacharykeskinen/scratch/data/uavsar')
ncs_dir = data_dir.joinpath('ncs')
tif_dir = data_dir.joinpath('tifs', 'Lowman, CO')
geom_dir = ncs_dir.joinpath('geometry')
tree_dir = data_dir.joinpath('trees')
nlcd = tree_dir.joinpath('nlcd')
biomass = tree_dir.joinpath('biomass')

In [3]:
ds = xr.open_dataset(ncs_dir.joinpath('tmp_uv_geom_atm_veg.nc'))

In [26]:
ds.to_netcdf(ncs_dir.joinpath('tmp_uv_geom_atm_veg.nc'))

MemoryError: Unable to allocate 30.4 GiB for an array with shape (9, 17017, 26631) and data type float64